In [1]:
!nvidia-smi

Thu May 23 07:24:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split




### Set up device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
# Setup manual transforms

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
# get food101 data
dataset = datasets.CIFAR10(root='./data', transform=manual_transforms, download=True)

100%|██████████| 170498071/170498071 [00:13<00:00, 13045154.89it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
### Check len of our dataset

len(dataset)

50000

In [5]:
## Splitting data into train, validation and test

train = int(0.8 * len(dataset))
validation = int(0.1 * len(dataset))
test = int(0.1 * len(dataset))

train_data, validation_data, test_data = random_split(dataset, [train, validation, test])

In [6]:
## preparing dataloaders

train_dataloader = DataLoader(train_data, batch_size = 16, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=False)

In [7]:
## Loading the pretained model and modifying the classfier layer's out_features

model = models.efficientnet_b0()
model = model.to(device)


model.classifier[1] = nn.Linear(model.classifier[1].in_features, out_features=10)

In [8]:
## setting up loss_function and optimizer n accuracy function

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)

def accuracy_function(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [9]:
## setting up training function and validation function
from tqdm.auto import tqdm
def train_function(model, device, train_dataloader, loss_function, optimizer):
  model.train()
  model = model.to(device)
  training_loss = 0
  for X_train, y_train in tqdm(train_dataloader):
    X_train, y_train = X_train.to(device), y_train.to(device)
    y_train_logits = model(X_train)
    loss = loss_function(y_train_logits, y_train)
    training_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  # find average loss per batch
  return (training_loss/len(train_dataloader))


def validate_function(model, device, validation_dataloader, loss_function, accuracy_function):
  model.eval()
  model = model.to(device)
  validation_loss = 0
  validation_acc = 0

  with torch.inference_mode():
    for X_val, y_val in tqdm(validation_dataloader):
      X_val , y_val = X_val.to(device), y_val.to(device)
      val_logits = model(X_val)
      val_pred = torch.argmax(val_logits, dim=1)
      validation_acc += accuracy_function(y_val, val_pred)
      validation_loss += loss_function(val_logits, y_val)
  return validation_loss/len(validation_dataloader), validation_acc/len(validation_dataloader)



In [10]:
## Actually training and validating the model

epochs = 20
for epoch in range(epochs):
  train_loss = train_function(model, device, train_dataloader, loss_function, optimizer)
  validation_loss, validation_acc = validate_function(model, device, validation_dataloader, loss_function, accuracy_function)
  print(f'Epoch {epoch}: Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_acc:.2f}%')

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 0: Training Loss: 2.0400, Validation Loss: 1.7901, Validation Accuracy: 34.27%


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

...

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 18: Training Loss: 0.5912, Validation Loss: 0.6265, Validation Accuracy: 79.11%


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 19: Training Loss: 0.5606, Validation Loss: 0.5973, Validation Accuracy: 79.73%
